### Connect to Cognite with OIDC

CDF UI:
- https://hafslundeco.fusion.cognite.com/
- heco-dev

Cognite Python SDK:
- https://cognite-docs.readthedocs-hosted.com/projects/cognite-sdk-python/en/latest/
- `pip install cognite-sdk msal`


In [118]:
import os
import pandas as pd
import numpy as np

from datetime import datetime
from msal import PublicClientApplication

from cognite.client import CogniteClient
from cognite.client.data_classes import TimeSeries, Asset

# Log-in detaljer
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
CDF_CLUSTER = "az-power-no-northeurope"
COGNITE_PROJECT = "heco-dev"

# Code to log-in WIHTOUT client_secret
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000


def authenticate_azure():

    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)

    # interactive login - make sure you have http://localhost:port in Redirect URI in App Registration as type "Mobile and desktop applications"
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    return creds


creds = authenticate_azure()

client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-python-dev",
)

#### Create asset if necessary

In [106]:
# Oppretter asset til et datasett

external_id_dataset = "uc:006:ml_test"
asset_name = "SE4" ### ENDRE DETTE
client.assets.create([Asset(
    external_id=asset_name,
    name=asset_name,
    data_set_id=client.data_sets.retrieve(external_id=external_id_dataset).id,
)])

,external_id,name,data_set_id,metadata,id,created_time,last_updated_time,root_id
0,SE4,SE4,788450409833218,{},4454379955528251,1660116569448,1660116569448,4454379955528251


In [122]:
res = client.time_series.retrieve(external_id="SE2_consumption_per_15min")
res.unit = "MWh/h"
res = client.time_series.update(res)

#### Add time series to data set

In [121]:
# Create a new timeseries connected to asset

timeseries = "SE2_consumption_per_15min" ## ENDRE DETTE
timeseries_ext_id = "SE2_consumption_per_15min"  ## ENDRE DETTE
asset_ext_id = 'SE2'  ## ENDRE DETTE


client.time_series.create(
    time_series=TimeSeries(
        external_id=timeseries_ext_id,
        asset_id=client.assets.retrieve(external_id=asset_ext_id).id,
        name=timeseries,
        unit="°C", ### ENDRE DETTE
        data_set_id=client.data_sets.retrieve(external_id="uc:006:ml_test").id
        )
    )


,value
id,8210339629314793
external_id,SE2_consumption_per_15min
name,SE2_consumption_per_15min
is_string,False
unit,°C
asset_id,6705865587929746
is_step,False
security_categories,[]
data_set_id,788450409833218
created_time,1660121343911


In [116]:
# Make a volue session

import wapi # wattsights egen pakke / den må installers via `pip install wapi-python`
import pandas as pd
import os
import numpy as np
from datetime import datetime, timezone
from datetime import timedelta
import pytz

# Create a session
# Dette forutsetter at dere har lagret volue credentials som miljøvariabler på PC'en deres og med navnet `WS_CLIENT_ID` og `WS_CLIENT_SECRET`
user = os.getenv("WS_CLIENT_ID")
password = os.getenv("WS_CLIENT_SECRET")
session = wapi.Session(client_id=user, client_secret=password)


In [123]:
# Extract data from Volue Insight

start = '2013-01-01'  #### ENDRE DETTE 
end = '2022-06-29'    #### ENDRE DETTE 
periods=pd.date_range(start,end,freq='D')
timezone_NO1 = pytz.timezone("Europe/Oslo")

print(periods)

curves = [
'con se1 mwh/h cet min15 a',
# 'con no1 ec00 mwh/h cet min15 f',
# 'pri no1 spot ec00 €/mwh cet h f',
# 'tt se2 con ec00 °c cet min15 f'
# 'cc se2 con ec00 % cet min15 f'
# 'con se2 ec00 mwh/h cet min15 f'
]  #### ENDRE DETTE 
external_id_timeseries = "SE2_consumption_per_15min"  #### ENDRE DETTE 
datapoints = []
df_watt = pd.DataFrame()

for curve in curves:
    print(f"Working with {curve}")
    curve = session.get_curve(name=curve)
    curve_type = curve.curve_type
    res = []
    counter = 1
    if curve_type == 'TIME_SERIES':
        ts = curve.get_data(data_from=start, data_to=end, function='SAVERAGE', frequency='MIN15')
        watt_temp = ts.to_pandas() # data gjort om til pandas

        df_watt = pd.merge(df_watt, watt_temp, left_index=True, right_index=True, how='outer')

        #df_watt.reset_index().rename(columns={'index':'date'})

        continue
   
    for j in range(len(periods)):
        if j == len(periods)-1:
            continue
        if curve_type == 'TAGGED_INSTANCES':
            ts_list = curve.search_instances(issue_date_from=periods[j],
                                 issue_date_to=periods[j+1],
                                 with_data=True, function='SAVERAGE', frequency='MIN15', tags=['Avg'])
        
        else: #curve_type == 'INSTANCES'
            ts_list = curve.search_instances(issue_date_from=periods[j],
                                 issue_date_to=periods[j+1],
                                 with_data=True, function='SAVERAGE', frequency='MIN15')
        # check if ts_list is empty and if yes, jump over that timestamp
        if not ts_list:
            continue
        pd_s = ts_list[0].to_pandas()
        if pd_s.empty:
            continue
        pd_s = pd_s.sort_index(ascending=True)
        pd_s = pd_s[:24]
        res.append(pd_s)
        print(f'Done {counter} out of {len(periods)}')
        counter += 1

    for k in range(len(res)):
        if k == 0:
            watt = pd.Series(res[k])
        else:
            watt = pd.concat([watt, res[k]])
    watt_temp = watt.rename(curve)
    df_watt = pd.merge(df_watt, watt_temp, left_index=True, right_index=True, how='outer')

df_watt.columns = ["price_forecast"]
#df_watt['Temp'] = df_watt.index
#df_watt['localizedtimestamp'] = df_watt.apply(lambda row : row['Temp'].replace(tzinfo=timezone).timestamp(), axis = 1)
#df_watt['timestamp'] = pd.to_datetime(df_watt.index).strftime('%Y-%m-%dT%H:%M:%SZ')
#print(df_watt.index[0])
#df_watt['Temp'][h] = df_watt['Temp'][h].replace(tzinfo=timezone.utc).timestamp()

#df_watt.dropna(inplace=True)
print(df_watt)


DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08',
               '2013-01-09', '2013-01-10',
               ...
               '2022-06-20', '2022-06-21', '2022-06-22', '2022-06-23',
               '2022-06-24', '2022-06-25', '2022-06-26', '2022-06-27',
               '2022-06-28', '2022-06-29'],
              dtype='datetime64[ns]', length=3467, freq='D')
Working with con se1 mwh/h cet min15 a
                           price_forecast
2013-01-01 00:00:00+01:00     1169.392301
2013-01-01 00:15:00+01:00     1169.392301
2013-01-01 00:30:00+01:00     1169.392301
2013-01-01 00:45:00+01:00     1169.392301
2013-01-01 01:00:00+01:00     1017.744309
...                                   ...
2022-06-28 22:45:00+02:00     1142.417000
2022-06-28 23:00:00+02:00      926.772000
2022-06-28 23:15:00+02:00      926.772000
2022-06-28 23:30:00+02:00      926.772000
2022-06-28 23:45:00+02:00      926.772000

[3327

ValueError: Out of range float values are not JSON compliant. Make sure your data does not contain NaN(s) or +/- Inf!

In [124]:
df_watt.dropna()
# df_watt
for h in range (len(df_watt)):


    date = datetime.timestamp(df_watt.index[h])
    date = datetime.fromtimestamp(date)
    date = timezone_NO1.localize(date)
    date = date.astimezone(pytz.UTC)

    #date = date.replace(tzinfo=None)

    point= (date, df_watt["price_forecast"].iloc[h])
    datapoints.append(point)

datapoints = [ (time.timestamp() * 1000, value) for time, value in datapoints]
# print(datapoints)

client.datapoints.insert(datapoints, id=client.time_series.retrieve(external_id=external_id_timeseries).id)

AttributeError: 'float' object has no attribute 'timestamp'

In [ ]:
for curve in curves:
    print(f"Working with {curve}")
    curve = session.get_curve(name=curve)
    curve_type = curve.curve_type
    res = []
    counter = 1
    if curve_type == 'TIME_SERIES':
        ts = curve.get_data(data_from=start, data_to=end, function='SAVERAGE', frequency='H')
        watt_temp = ts.to_pandas() # data gjort om til pandas

        df_watt = pd.merge(df_watt, watt_temp, left_index=True, right_index=True, how='outer')

        #df_watt.reset_index().rename(columns={'index':'date'})

        continue
   
    for j in range(len(periods)):
        if j == len(periods)-1:
            continue
        if curve_type == 'TAGGED_INSTANCES':
            ts_list = curve.search_instances(issue_date_from=periods[j],
                                 issue_date_to=periods[j+1],
                                 with_data=True, function='SAVERAGE', frequency='H', tags=['Avg'])
        else: #curve_type == 'INSTANCES'
            ts_list = curve.search_instances(issue_date_from=periods[j],
                                 issue_date_to=periods[j+1],
                                 with_data=True, function='SAVERAGE', frequency='H')
        # check if ts_list is empty and if yes, jump over that timestamp
        if not ts_list:
            continue
        pd_s = ts_list[0].to_pandas()
        if pd_s.empty:
            continue
        pd_s = pd_s.sort_index(ascending=True)
        pd_s = pd_s[:24]
        res.append(pd_s)
        print(f'Done {counter} out of {len(periods)}')
        counter += 1

    for k in range(len(res)):
        if k == 0:
            watt = pd.Series(res[k])
        else:
            watt = pd.concat([watt, res[k]])
    watt_temp = watt.rename(curve)
    df_watt = pd.merge(df_watt, watt_temp, left_index=True, right_index=True, how='outer')